# Convert KML to Data Frame and CSV

In [9]:
import xml.etree.ElementTree as ET
import pandas as pd

# Define namespaces
namespace = {
    "kml": "http://www.opengis.net/kml/2.2",
    "gx": "http://www.google.com/kml/ext/2.2",
}

def parse_kml_to_dataframe(kml_file_path):
    """
    Parses a KML file and returns a DataFrame with track and extended data.

    Parameters:
    - kml_file_path (str): Path to the KML file to parse.

    Returns:
    - pd.DataFrame: DataFrame containing the parsed track and extended data.
    """
    # Parse the XML file
    tree = ET.parse(kml_file_path)
    root = tree.getroot()

    # Initialize a list to store the track data
    track_data = []

    # Extract extended data if present (assuming a single ExtendedData for the document)
    extended_values = {}
    extended_data = root.find(".//kml:ExtendedData/kml:SchemaData", namespaces=namespace)
    if extended_data is not None:
        for array_data in extended_data.findall("gx:SimpleArrayData", namespaces=namespace):
            name = array_data.get("name")
            values = [float(value.text) for value in array_data.findall("gx:value", namespaces=namespace)]
            extended_values[name] = values

    # Find all Placemark elements with gx:Track children
    for placemark in root.findall(".//kml:Placemark", namespaces=namespace):
        track = placemark.find("gx:Track", namespaces=namespace)
        if track is not None:
            # Extract timestamps and coordinates
            timestamps = [when.text for when in track.findall("kml:when", namespaces=namespace)]
            coords = [coord.text for coord in track.findall("gx:coord", namespaces=namespace)]

            # Pair each timestamp with its corresponding coordinates and indexed extended data
            for index, (time, coord) in enumerate(zip(timestamps, coords)):
                lon, lat, alt = map(float, coord.split())
                data_entry = {
                    "timestamp": time,
                    "latitude": lat,
                    "longitude": lon,
                    "altitude_m": alt,
                    "acc_horiz": extended_values.get("acc_horiz", [None])[index] if "acc_horiz" in extended_values else None,
                    "acc_vert": extended_values.get("acc_vert", [None])[index] if "acc_vert" in extended_values else None,
                    "course": extended_values.get("course", [None])[index] if "course" in extended_values else None,
                    "speed_kts": extended_values.get("speed_kts", [None])[index] if "speed_kts" in extended_values else None,
                    "altitude_ft": extended_values.get("altitude", [None])[index] if "altitude" in extended_values else None,
                    "bank": extended_values.get("bank", [None])[index] if "bank" in extended_values else None,
                    "pitch": extended_values.get("pitch", [None])[index] if "pitch" in extended_values else None,
                }
                track_data.append(data_entry)

    # Convert to a DataFrame
    df = pd.DataFrame(track_data)
    return df

# List of KML file paths
file_paths = ["./data/kml/5Nov24-flight1.kml", "./data/kml/6Nov24-flight2.kml", "./data/kml/6Nov24-flight3.kml", "./data/kml/6Nov24-flight4.kml", "./data/kml/7Nov24-flight5.kml"]
for file_path in file_paths:
    df = parse_kml_to_dataframe(file_path)
    output_csv_path = file_path.replace(".kml", ".csv").replace("/kml/", "/csv/")
    df.to_csv(output_csv_path, index=False)
    print(f"Data from {file_path} saved to {output_csv_path}")


Data from ./data/kml/5Nov24-flight1.kml saved to ./data/csv/5Nov24-flight1.csv
Data from ./data/kml/6Nov24-flight2.kml saved to ./data/csv/6Nov24-flight2.csv
Data from ./data/kml/6Nov24-flight3.kml saved to ./data/csv/6Nov24-flight3.csv
Data from ./data/kml/6Nov24-flight4.kml saved to ./data/csv/6Nov24-flight4.csv
Data from ./data/kml/7Nov24-flight5.kml saved to ./data/csv/7Nov24-flight5.csv


In [10]:
# # Display or save the DataFrame
# df